### Feature engineering

For each Gitcoin round we generated all the pairs of donated wallets.
Then we calculated the following aggregations for the different time periods. 

- Aggregations:
    - Tenure - how many days from the first to the end of period
    - Frequency - number of transactions
    - Monetary - transactions amount
    - Consistency - number of active unique days
- Filters:
    - All Ethereum transactions for the last year till round ends (_year)
    - All Ethereum transactions for the round period (_round)

In [1]:
# Rounds dataset
df = pd.read_csv('data/data_rounds.csv').rename(columns = {'source_wallet': 'wallet'})
df_rounds = df.groupby('round_name')['created_ts'].agg([min,max])

# Transactions
dir1 = 'data/external/v2/trnx/'
files = [f for f in os.listdir(dir1) if f[-3:] == 'csv']
df_feats = pd.DataFrame()

for file in tqdm(files):
    df = pd.read_csv(f'data/external/v2/trnx/{file}')
    df['day'] = df['dt'].str[:10]
    for feat_round in df_rounds.index:
        # parameters
        t1 = df_rounds.loc[feat_round]['min']
        t2 = df_rounds.loc[feat_round]['max']
        t0 = t2 - 60*60*24*365*1

        # filters for the dataset
        f0 = (df.is_error == 0) & (df.wallet != df.wallet_add)
        f11 = df[f'flg_{feat_round}'] == 1
        f12 = df[f'flg_add_{feat_round}'] == 1
        f20 = (df.timestamp > t0) & (df.timestamp < t2)
        f21 = (df.timestamp > t1) & (df.timestamp < t2)
        list_filters = [
            ('year', f0&f11&f12&f20),
            ('round',f0&f11&f12&f21),
        ]
        for feat_filt,filt in list_filters:
            # aggregations
            df1 = df[filt].groupby(['wallet','wallet_add']).agg({
                'timestamp': min,
                'value': [len,sum],
                'day': lambda x: len(set(x)),
            })
            df1.columns = ['tenure','frequency','monetary','consistency']
            df1['tenure'] = (t2 - df1.tenure)/(60*60*24)
            df1.reset_index(inplace = True)
            # collecting
            df1['round_name'] = feat_round
            df1['filt'] = feat_filt
            df_feats = pd.concat([df_feats, df1])
            
# transpose
feats_gr = ['round_name','wallet','wallet_add','filt']
feats_ag = ['tenure','frequency','monetary','consistency']
df_feats2 = df_feats.groupby(feats_gr)[feats_ag].min().unstack()
cols = df_feats2.columns
df_feats2.columns = ['_'.join(c) for c in cols]
df_feats2.reset_index(inplace = True)

# create wallets hash
df_feats2['wallets'] = df_feats2[['wallet','wallet_add']].apply(lambda x: '_'.join(sorted(x)), axis = 1)
del df_feats2['wallet']
del df_feats2['wallet_add']
df_feats = df_feats2.groupby(['round_name','wallets']).first().reset_index()

100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [00:39<00:00,  1.56s/it]


In [2]:
# Save results
feats_info = ['round_name','wallets']
feats_agg  = ['tenure','frequency','monetary','consistency']
feats_filt = ['year','round']

feats1 = [f'{feat}_{feats_filt[0]}' for feat in feats_agg]
feats2 = [f'{feat}_{feats_filt[1]}' for feat in feats_agg]
feats = feats1 + feats2

df_feats = df_feats[feats_info + feats]
df_feats.to_csv('data/data_features.csv', index = False)
print(df_feats.shape)
df_feats[:5]

(37612, 10)


,round_name,wallets,tenure_year,frequency_year,monetary_year,consistency_year,tenure_round,frequency_round,monetary_round,consistency_round
0,1_gitcoin_gr15,0x0000ce08fa224696a819877070bf378e8b131acf_0xa...,297.27,2.00,0.21,1.00,NaN,NaN,NaN,NaN
1,1_gitcoin_gr15,0x0003a2d21b35c7cfc0fb259c9e27dbdb434864bd_0x3...,31.70,1.00,0.01,1.00,NaN,NaN,NaN,NaN
2,1_gitcoin_gr15,0x0003a2d21b35c7cfc0fb259c9e27dbdb434864bd_0x4...,274.64,1.00,0.04,1.00,NaN,NaN,NaN,NaN
3,1_gitcoin_gr15,0x0003a2d21b35c7cfc0fb259c9e27dbdb434864bd_0x4...,274.63,1.00,0.02,1.00,NaN,NaN,NaN,NaN
4,1_gitcoin_gr15,0x00041f83818286276bd5a7507088b7c4dff1c5a4_0xa...,62.99,1.00,0.63,1.00,NaN,NaN,NaN,NaN
